# Preparation of Aviation Data

# Load all necessary packages

In [1]:
import xml.etree.ElementTree as et
import json
import pandas as pd
import glob
import numpy as np

# Load and Parse XML Aviation Data File

In [2]:
file = 'Documents/Spring_2020/data-scientist-exercise02/data/AviationData.xml'

In [3]:
#Uncomment and run to get column names
#tree = et.parse(file)
#root = tree.getroot()
#for elem in root:
    #for subelem in elem:
        #print(subelem.attrib)

In [4]:
col_names = ['EventId','InvestigationType','AccidentNumber','EventDate','Location','Country','Latitude','Longitude','AirportCode',
    'AirportName','InjurySeverity','AircraftDamage','AircraftCategory','RegistrationNumber','Make','Model','AmateurBuilt','NumberOfEngines',
    'EngineType','FARDescription','Schedule','PurposeOfFlight','AirCarrier','TotalFatalInjuries','TotalSeriousInjuries','TotalMinorInjuries',
    'TotalUninjured','WeatherCondition','BroadPhaseOfFlight','ReportStatus','PublicationDate']


In [5]:
def parse_XML(xml_file, df_cols): 
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    for node in xroot:
        for subnode in node:
            res = []
            for i in range(len(df_cols)):
                res.append(subnode.attrib.get(df_cols[i]))
            rows.append({df_cols[i]: res[i] for i, _ in enumerate(df_cols)})

    out_df = pd.DataFrame(rows, columns=df_cols)

    return out_df

In [6]:
parsed_xml = parse_XML(file,col_names)

# Load and Merge JSON Files

In [7]:
file_list = glob.glob("Documents/Spring_2020/data-scientist-exercise02/data/*.json")

In [8]:
allFilesDict = {v:k for v, k in enumerate(file_list, 1)}

data = []

for k,v in allFilesDict.items():
    with open(v, 'r') as d:
        jdata = json.load(d)
        if jdata:
            data.append(jdata)

data_list = []
for i in range(0,len(data)):
    data_1_file = data[i]
    data_1_file = data_1_file['data']
    df = pd.DataFrame(data_1_file)
    data_list.append(df)

json_data = pd.concat(data_list)

In [9]:
combined_data = pd.merge(parsed_xml, json_data, on='EventId', how='left')

# Explore, Clean, and Enrich Data

In [11]:
combined_data = combined_data.replace('',np.nan)

In [12]:
#create fatal indicator column to indicate if accident was deadly
combined_data['TotalFatalInjuries'] = combined_data['TotalFatalInjuries'].astype(float)
combined_data['FatalIndicator'] = combined_data['TotalFatalInjuries'].apply(lambda x: 1 if x > 0 else 0)
combined_data['FatalIndicator'].value_counts()

0    61829
1    15428
Name: FatalIndicator, dtype: int64

In [13]:
#Drop Accidents with no event date- only 4 observations
#Create fields for year, month, day, and day of week
combined_data = combined_data.dropna(subset= ['EventDate'])
combined_data['Year'] = pd.DatetimeIndex(combined_data['EventDate']).year.astype(int)
combined_data['Month'] = pd.DatetimeIndex(combined_data['EventDate']).month.astype(int)
combined_data['Day'] = pd.DatetimeIndex(combined_data['EventDate']).day.astype(int)
combined_data['WeekDay'] = pd.to_datetime(combined_data['EventDate']).apply(lambda x: x.weekday()).astype(int)

In [14]:
combined_data.to_csv('Downloads/combined_aviation_data.csv')